# Deep Dive into LangChain

## import the openAI key

In [3]:
from dotenv import load_dotenv
import os
load_dotenv("./.env")
API_KEY = os.getenv("API_OAI_KEY")

## ChatModels: GPT-3.5-Turbo and GPT-4

In [7]:
from langchain_openai import ChatOpenAI

prompt = "Explain Quantum Mechanics in one sentence"

llm = ChatOpenAI(api_key=API_KEY, model="gpt-4o-mini")
output = llm.invoke(prompt)
print(output.content)

Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy at the smallest scales, where particles exhibit wave-particle duality and uncertainty principles, leading to phenomena that defy classical intuition.


In [8]:
# Defining the roles using LangChain
from langchain.schema import (
	SystemMessage,
	AIMessage,
	HumanMessage

)

In [9]:
messages = [
	SystemMessage(content="Your are a physisit and respond only in German"),
	HumanMessage(content="Explain Quantum Mechanics in one sentence")
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik ist die Theorie, die das Verhalten von Materie und Energie auf der kleinsten Skala, der atomaren und subatomaren Ebene, beschreibt und dabei Konzepte wie Superposition und Verschränkung einführt.
